# Enkel bildeklassifisering

Denne notatboken viser deg hvordan du kan klassifisere bilder ved hjelp av et forhåndstrent nevralt nettverk.

**Hva du vil lære:**
- Hvordan laste inn og bruke en forhåndstrent modell
- Forbehandling av bilder
- Å gjøre prediksjoner på bilder
- Forståelse av tillitsnivåer

**Bruksområde:** Identifisere objekter i bilder (som "katt", "hund", "bil", osv.)

---


## Steg 1: Importer nødvendige biblioteker

La oss importere verktøyene vi trenger. Ikke bekymre deg hvis du ikke forstår alle disse ennå!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Steg 2: Last inn forhåndstrent modell

Vi skal bruke **MobileNetV2**, et nevralt nettverk som allerede er trent på millioner av bilder.

Dette kalles **Transfer Learning** - å bruke en modell som noen andre har trent!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Trinn 3: Hjelpefunksjoner

La oss lage funksjoner for å laste inn og klargjøre bilder for modellen vår.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Steg 4: Test på eksempelbilder

La oss prøve å klassifisere noen bilder fra internett!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Klassifiser Hvert Bilde


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Steg 5: Prøv dine egne bilder!

Bytt ut URL-en nedenfor med hvilken som helst bilde-URL du ønsker å klassifisere.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Hva skjedde akkurat?

1. **Vi lastet inn en forhåndstrent modell** - MobileNetV2 ble trent på millioner av bilder
2. **Vi forhåndsprosesserte bilder** - Endret størrelse og format for modellen
3. **Modellen gjorde prediksjoner** - Den ga sannsynligheter for 1000 objektklasser
4. **Vi dekodet resultatene** - Konverterte tall til lesbare etiketter

### Forståelse av tillitsnivåer

- **90-100%**: Veldig sikker (nesten garantert riktig)
- **70-90%**: Sikker (sannsynligvis riktig)
- **50-70%**: Noe sikker (kan være riktig)
- **Under 50%**: Ikke veldig sikker (usikker)

### Hvorfor kan prediksjoner være feil?

- **Uvanlig vinkel eller belysning** - Modellen ble trent på typiske bilder
- **Flere objekter** - Modellen forventer ett hovedobjekt
- **Sjeldne objekter** - Modellen kjenner bare 1000 kategorier
- **Lav bildekvalitet** - Uklare eller pikselerte bilder er vanskeligere


## 🚀 Neste steg

1. **Prøv forskjellige bilder:**
   - Finn bilder på [Unsplash](https://unsplash.com)
   - Høyreklikk → "Kopier bildeadresse" for å få URL

2. **Eksperimenter:**
   - Hva skjer med abstrakt kunst?
   - Kan den gjenkjenne objekter fra forskjellige vinkler?
   - Hvordan håndterer den flere objekter?

3. **Lær mer:**
   - Utforsk [Computer Vision-leksjoner](../lessons/4-ComputerVision/README.md)
   - Lær å trene din egen bildegjenkjenner
   - Forstå hvordan CNN-er (Convolutional Neural Networks) fungerer

---

## 🎉 Gratulerer!

Du har nettopp bygget en bildegjenkjenner ved hjelp av et toppmoderne nevralt nettverk!

Denne teknikken brukes til:
- Google Photos (organisering av bilder)
- Selvkjørende biler (gjenkjenning av objekter)
- Medisinsk diagnostikk (analyse av røntgenbilder)
- Kvalitetskontroll (oppdage feil)

Fortsett å utforske og lære! 🚀



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi tilstreber nøyaktighet, vennligst vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør betraktes som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for eventuelle misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
